In [126]:
# Probably more imports than are really necessary...
import os
import torch
import torchaudio
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn as nn
import torch.nn.functional as F
from torchaudio.transforms import MelSpectrogram, AmplitudeToDB
from tqdm import tqdm
import librosa
import numpy as np
import miditoolkit
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, average_precision_score, accuracy_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from xgboost import XGBClassifier
import xgboost as xgb

import random

## Metrics

In [3]:
def accuracy1(groundtruth, predictions):
    correct = 0
    for k in groundtruth:
        if not (k in predictions):
            print("Missing " + str(k) + " from predictions")
            return 0
        if predictions[k] == groundtruth[k]:
            correct += 1
    return correct / len(groundtruth)

In [4]:
def accuracy2(groundtruth, predictions):
    correct = 0
    for k in groundtruth:
        if not (k in predictions):
            print("Missing " + str(k) + " from predictions")
            return 0
        if predictions[k] == groundtruth[k]:
            correct += 1
    return correct / len(groundtruth)

In [6]:
TAGS = ['rock', 'oldies', 'jazz', 'pop', 'dance',  'blues',  'punk', 'chill', 'electronic', 'country']

In [10]:
def accuracy3(groundtruth, predictions):
    preds, targets = [], []
    for k in groundtruth:
        if not (k in predictions):
            print("Missing " + str(k) + " from predictions")
            return 0
        prediction = [1 if tag in predictions[k] else 0 for tag in TAGS]
        target = [1 if tag in groundtruth[k] else 0 for tag in TAGS]
        preds.append(prediction)
        targets.append(target)
    
    mAP = average_precision_score(targets, preds, average='macro')
    return mAP

## Task 1: Composer classification

In [13]:
dataroot1 = "student_files/task1_composer_classification/"

In [45]:
class model1():
    def __init__(self):
        # pass
        self.scaler = None
        self.le = None
        self.model = None

    def features(self, path):
        filepath = path if os.path.exists(path) else os.path.join(dataroot1, path)
        midi_obj = miditoolkit.midi.parser.MidiFile(filepath)
        # midi_obj = miditoolkit.midi.parser.MidiFile(dataroot1 + '/' + path)
        notes = midi_obj.instruments[0].notes
        num_notes = len(notes)
        
        # avoid divide by zero
        if num_notes == 0:
            return [0.0, 0.0]

        pitches = [note.pitch for note in notes]
        durations = [note.end - note.start for note in notes]
        total_time = max(note.end for note in notes) - min(note.start for note in notes)
        
        average_pitch = sum([note.pitch for note in notes]) / num_notes
        std_pitch = np.std([note.pitch for note in notes])
        average_duration = sum([note.end - note.start for note in notes]) / num_notes
        std_duration = np.std(durations)
        note_density = num_notes / total_time if total_time > 0 else 0
        pitch_range = max(pitches) - min(pitches)

        # Pitch class histogram
        pitch_classes = [note.pitch % 12 for note in notes]
        hist = [0] * 12
        for pc in pitch_classes:
            hist[pc] += 1
        hist = [x / num_notes for x in hist] # normalize

        # interval histogram
        intervals = [pitches[i+1] - pitches[i] for i in range(len(pitches)-1)]
        hist_intervals = [0] * 5
        for interval in intervals:
            if interval < -6:
                hist_intervals[0] += 1
            elif interval < 0:
                hist_intervals[1] += 1
            elif interval == 0:
                hist_intervals[2] += 1
            elif interval <= 6:
                hist_intervals[3] += 1
            else:
                hist_intervals[4] += 1

        if intervals:
            hist_intervals = [x / len(intervals) for x in hist_intervals]
        else:
            hist_intervals = [0] * 5


        # symbolic features
        unique_pitches = len(set(pitches))
        start_times = [note.start for note in notes]
        unique_starts = len(set(start_times))
        articulation_rate = unique_starts / total_time if total_time > 0 else 0

        sorted_notes = sorted(notes, key=lambda n: n.start)
        rest_time = 0.0
        for i in range(1, len(sorted_notes)):
            prev_end = sorted_notes[i-1].end
            curr_start = sorted_notes[i].start
            if curr_start > prev_end:
                rest_time += curr_start - prev_end
        rest_ratio = rest_time / total_time if total_time > 0 else 0

        # polyphony
        start_time_counts = {}
        for note in notes:
            t = round(note.start, 3)
            start_time_counts[t] = start_time_counts.get(t,0) + 1
        polyphonic_events = sum(1 for count in start_time_counts.values() if count > 1)
        polyphony_ratio = polyphonic_events / len(start_time_counts) if start_time_counts else 0

        # unique duration count
        unique_durations = len(set(round(d, 3) for d in durations))

        # longest rest
        longest_rest = 0.0
        for i in range(1, len(sorted_notes)):
            gap = sorted_notes[i].start - sorted_notes[i-1].end
            if gap > longest_rest:
                longest_rest = gap


        # note start times
        onset_std = np.std(start_times) if len(start_times) > 1 else 0

        # notes per beat
        notes_per_beat = num_notes / midi_obj.ticks_per_beat if midi_obj.ticks_per_beat > 0 else 0

        
        # combined feature vector
        features = [average_pitch, std_pitch, average_duration,
                    std_duration, note_density, pitch_range] + hist + hist_intervals + [
                        unique_pitches, articulation_rate, rest_ratio, 
                        polyphony_ratio, unique_durations, longest_rest,
                        onset_std, notes_per_beat]
        
        return features
    
    def predict(self, path, outpath=None):
        d = eval(open(path, 'r').read())
        predictions = {}
        for k in d:
            x = self.features(k)
            x_scaled = self.scaler.transform([x])
            raw = self.model.predict(x_scaled)
            # pred = self.model.predict([x])
            label = self.le.inverse_transform(raw)[0]
            predictions[k] = label
        if outpath:
            with open(outpath, "w") as z:
                # z.write(str(predictions) + '\n')
                json.dump(predictions, z)
        return predictions

    # Train your model. Note that this function will not be called from the autograder:
    # instead you should upload your saved model using save()
    def train(self, path):
        with open(path, 'r') as f:
            train_json = eval(f.read())

        # build feature matrix w labels
        X = [self.features(k) for k in train_json]
        y = [train_json[k] for k in train_json]

        # label encode composer names
        self.le = LabelEncoder()
        y_enc = self.le.fit_transform(y)

        # scale features
        self.scaler = StandardScaler()
        X_scaled = self.scaler.fit_transform(X)

        # train/val split
        X_tr, X_val, y_tr, y_val = train_test_split(
            X_scaled, y_enc, 
            test_size=0.2, 
            stratify=y_enc, 
            random_state=42
        )

        
        # fit XGBoost
        self.model = XGBClassifier(
            use_label_encoder=False,
            objective='multi:softprob',
            eval_metric='mlogloss',
            n_estimators=500,
            max_depth=8,
            learning_rate=0.05,
            subsample=0.8,
            colsample_bytree=0.8,
            gamma=1.0,
            min_child_weight=1,
            reg_alpha=0.1,
            reg_lambda=1.0,
            random_state=42
        )

        dtrain_full = xgb.DMatrix(X_scaled, label=y_enc)
        params = self.model.get_xgb_params()
        params['num_class'] = len(np.unique(y_enc))
        cv_results = xgb.cv(
            params=params,
            dtrain=dtrain_full,
            num_boost_round=self.model.get_params()['n_estimators'],
            nfold=5,
            metrics='mlogloss',
            early_stopping_rounds=20,
            seed=42,
            verbose_eval=True
        )
        best_n = len(cv_results)

        # refit on full training data w optimal num of trees
        self.model.set_params(n_estimators=best_n)
        self.model.fit(X_scaled, y_enc)
        
        # self.model.fit(
        #     X_tr, y_tr,
        #     eval_set=[(X_val, y_val)],
        #     verbose=True
        # )
        
        # model.fit(X_train, y_train)
        # self.model = model
        # X_train = [self.features(k) for k in train_json]
        # y_train = [train_json[k] for k in train_json]
        # model = LogisticRegression(max_iter=1000)

## Task 2: Sequence prediction

In [60]:
dataroot2 = "student_files/task2_next_sequence_prediction/"

In [128]:
class model2():
    def __init__(self):
        self.scaler = None
        self.model = XGBClassifier(
            use_label_encoder=False,
            objective='binary:logistic',
            eval_metric='logloss',
            
            n_estimators=1000,
            max_depth=8,
            learning_rate=0.02,
            
            subsample=0.7,
            colsample_bytree=0.7,

            gamma=2.0,
            min_child_weight=5,

            reg_alpha=1.0,
            reg_lambda=1.0,

            tree_method='hist',
            random_state=42
        )

        self.base_rf = RandomForestClassifier(
            n_estimators=200,
            max_depth=10,
            random_state=42
        )

    # stack models
        self.model = StackingClassifier(
            estimators=[
                ('xgb', self.base_xgb),
                ('rf', self.base_rf),
            ],
            final_estimator=LogisticRegression(max_iter=1000),
            cv=5,
            n_jobs=-1
        )

    @staticmethod
    def _js_divergence(p, q):
        p, q = np.array(p), np.array(q)
        m = 0.5 * (p + q)
        return 0.5 * (entropy(p, m) + entropy(q, m))

    @staticmethod
    def _pearson_corr(p, q):
        p, q = np.array(p), np.array(q)
        if p.std() == 0 or q.std() == 0:
            return 0.0
        return np.corrcoef(p, q)[0, 1]


    def features(self, path):
        # midi_obj = miditoolkit.midi.parser.MidiFile(dataroot2 + '/' + path)
        filepath = path if os.path.exists(path) else os.path.join(dataroot2, path)
        midi_obj = miditoolkit.midi.parser.MidiFile(filepath)
        notes = midi_obj.instruments[0].notes
        num_notes = len(notes)
        
        if num_notes == 0:
            return [0]*31

        pitches = [note.pitch for note in notes]
        durations = [note.end - note.start for note in notes]
        total_time = max(note.end for note in notes) - min(note.start for note in notes)
        
        average_pitch = sum([note.pitch for note in notes]) / num_notes
        std_pitch = np.std([note.pitch for note in notes])
        average_duration = sum([note.end - note.start for note in notes]) / num_notes
        std_duration = np.std(durations)
        note_density = num_notes / total_time if total_time > 0 else 0
        pitch_range = max(pitches) - min(pitches)

        # Pitch class histogram
        pitch_classes = [note.pitch % 12 for note in notes]
        hist = [0] * 12
        for pc in pitch_classes:
            hist[pc] += 1
        hist = [x / num_notes for x in hist] # normalize

        # interval histogram
        intervals = [pitches[i+1] - pitches[i] for i in range(len(pitches)-1)]
        hist_intervals = [0] * 5
        for interval in intervals:
            if interval < -6:
                hist_intervals[0] += 1
            elif interval < 0:
                hist_intervals[1] += 1
            elif interval == 0:
                hist_intervals[2] += 1
            elif interval <= 6:
                hist_intervals[3] += 1
            else:
                hist_intervals[4] += 1

        if intervals:
            hist_intervals = [x / len(intervals) for x in hist_intervals]
        else:
            hist_intervals = [0] * 5


        # symbolic features
        unique_pitches = len(set(pitches))
        start_times = [note.start for note in notes]
        unique_starts = len(set(start_times))
        articulation_rate = unique_starts / total_time if total_time > 0 else 0

        sorted_notes = sorted(notes, key=lambda n: n.start)
        rest_time = 0.0
        for i in range(1, len(sorted_notes)):
            prev_end = sorted_notes[i-1].end
            curr_start = sorted_notes[i].start
            if curr_start > prev_end:
                rest_time += curr_start - prev_end
        rest_ratio = rest_time / total_time if total_time > 0 else 0

        # polyphony
        start_time_counts = {}
        for note in notes:
            t = round(note.start, 3)
            start_time_counts[t] = start_time_counts.get(t,0) + 1
        polyphonic_events = sum(1 for count in start_time_counts.values() if count > 1)
        polyphony_ratio = polyphonic_events / len(start_time_counts) if start_time_counts else 0

        # unique duration count
        unique_durations = len(set(round(d, 3) for d in durations))

        # longest rest
        longest_rest = 0.0
        for i in range(1, len(sorted_notes)):
            gap = sorted_notes[i].start - sorted_notes[i-1].end
            if gap > longest_rest:
                longest_rest = gap


        # note start times
        onset_std = np.std(start_times) if len(start_times) > 1 else 0

        # notes per beat
        notes_per_beat = num_notes / midi_obj.ticks_per_beat if midi_obj.ticks_per_beat > 0 else 0

        
        # combined feature vector
        features = [average_pitch, std_pitch, average_duration,
                    std_duration, note_density, pitch_range] + hist + hist_intervals + [
                        unique_pitches, articulation_rate, rest_ratio, 
                        polyphony_ratio, unique_durations, longest_rest,
                        onset_std, notes_per_beat]
        
        
        return features
    
    def train(self, path):
        # This baseline doesn't use any model (it just measures feature similarity)
        # You can use this approach but *probably* you'll want to implement a model
        with open(path,'r') as f:
            data = eval(f.read())

        # build x and y
        X_pairs, y = [], []
        for (b1,b2), label in data.items():
            f1 = self.features(b1)
            f2 = self.features(b2)
            # basic difference
            diff = [abs(a-b) for a,b in zip(f1,f2)]
            # elemnt product
            prod = [a*b for a,b in zip(f1,f2)]
            # squared dif
            sqdiff = [(a-b)**2 for a,b in zip(f1,f2)]
            # ratio
            ratio = [a/(b+1e-6) for a,b in zip(f1,f2)]
            
            # distributional similarities
            pc1, pc2 = f1[6:18], f2[6:18]
            iv1, iv2 = f1[18:23], f2[18:23]
            js_pc   = model2._js_divergence(pc1, pc2)
            corr_pc = model2._pearson_corr(pc1, pc2)
            js_iv   = model2._js_divergence(iv1, iv2)
            corr_iv = model2._pearson_corr(iv1, iv2)

            feat_vec = f1 + f2 + diff + prod + sqdiff + ratio + [js_pc, corr_pc, js_iv, corr_iv]
            X_pairs.append(feat_vec)
            y.append(int(label))

        # 3 scale and split
        self.scaler = StandardScaler()
        Xs = self.scaler.fit_transform(X_pairs)
        # X_tr, X_val, y_tr, y_val = train_test_split(
        #     Xs, y, test_size=0.2, stratify=y, random_state=42
        # )


        # 4) use xgb.cv to find best n_estimators
        dtrain_full = xgb.DMatrix(Xs, label=y)
        params = self.model.get_xgb_params()
        # esnure binary objective and metric
        params['objective'] = 'binary:logistic'
        params['eval_metric'] = 'logloss'

        cv_results = xgb.cv(
            params=params,
            dtrain=dtrain_full,
            num_boost_round=self.model.get_params()['n_estimators'],
            nfold=5,
            early_stopping_rounds=20,
            seed=42,
            verbose_eval=True
        )
        best_n = len(cv_results)

        # 5. refit on all training data w optimal number of trees
        self.model.set_params(n_estimators=best_n)
        self.model.fit(Xs,y)



    def predict(self, path, outpath=None):
        d = eval(open(path, 'r').read())
        predictions = {}
        for (b1,b2) in d:
            # Keys are pairs of paths
            f1 = self.features(b1)
            f2 = self.features(b2)

            diff = [abs(a-b) for a,b in zip(f1,f2)]
            prod = [a*b for a,b in zip(f1,f2)]
            sqdiff = [(a-b)**2 for a,b in zip(f1,f2)]
            ratio = [a/(b+1e-6) for a,b in zip(f1,f2)]
            # feature vector
            feat_vec = f1 + f2 + diff + prod + sqdiff + ratio
            pc1, pc2 = f1[6:18], f2[6:18]
            iv1, iv2 = f1[18:23], f2[18:23]
            js_pc   = model2._js_divergence(pc1, pc2)
            corr_pc = model2._pearson_corr(pc1, pc2)
            js_iv   = model2._js_divergence(iv1, iv2)
            corr_iv = model2._pearson_corr(iv1, iv2)

            feat_vec = f1 + f2 + diff + prod + sqdiff + ratio + [js_pc, corr_pc, js_iv, corr_iv]

            
            x = self.scaler.transform([feat_vec])
            p = self.model.predict(x)[0]
            predictions[(b1,b2)] = bool(p)
            
            # Note: hardcoded difference between features
            # if abs(x1[0] - x2[0]) < 5:
            #     predictions[k] = True
            # else:
            #     predictions[k] = False
        if outpath:
            with open(outpath, "w") as z:
                z.write(str(predictions) + '\n')
        return predictions

## Task 3: Audio classification

In [ ]:
# Some constants (you can change any of these if useful)
SAMPLE_RATE = 16000
N_MELS = 64
N_CLASSES = 10
AUDIO_DURATION = 10 # seconds
BATCH_SIZE = 32

In [ ]:
dataroot3 = "student_files/task3_audio_classification/"

In [ ]:
def extract_waveform(path):
    waveform, sr = librosa.load(dataroot3 + '/' + path, sr=SAMPLE_RATE)
    waveform = np.array([waveform])
    if sr != SAMPLE_RATE:
        resample = torchaudio.transforms.Resample(orig_freq=sr, new_freq=SAMPLE_RATE)
        waveform = resample(waveform)
    # Pad so that everything is the right length
    target_len = SAMPLE_RATE * AUDIO_DURATION
    if waveform.shape[1] < target_len:
        pad_len = target_len - waveform.shape[1]
        waveform = F.pad(waveform, (0, pad_len))
    else:
        waveform = waveform[:, :target_len]
    waveform = torch.FloatTensor(waveform)
    return waveform

In [ ]:
class AudioDataset(Dataset):
    def __init__(self, meta, preload = True):
        self.meta = meta
        ks = list(meta.keys())
        self.idToPath = dict(zip(range(len(ks)), ks))
        self.pathToFeat = {}

        self.mel = MelSpectrogram(sample_rate=SAMPLE_RATE, n_mels=N_MELS)
        self.db = AmplitudeToDB()
        
        self.preload = preload # Determines whether the features should be preloaded (uses more memory)
                               # or read from disk / computed each time (slow if your system is i/o-bound)
        if self.preload:
            for path in ks:
                waveform = extract_waveform(path)
                mel_spec = self.db(self.mel(waveform)).squeeze(0)
                self.pathToFeat[path] = mel_spec

    def __len__(self):
        return len(self.meta)
    
    def __getitem__(self, idx):
        # Faster version, preloads the features
        path = self.idToPath[idx]
        tags = self.meta[path]
        bin_label = torch.tensor([1 if tag in tags else 0 for tag in TAGS], dtype=torch.float32)

        if self.preload:
            mel_spec = self.pathToFeat[path]
        else:
            waveform = extract_waveform(path)
            mel_spec = self.db(self.mel(waveform)).squeeze(0)
        
        return mel_spec.unsqueeze(0), bin_label, path

In [ ]:
class Loaders():
    def __init__(self, train_path, test_path, split_ratio=0.9, seed = 0):
        torch.manual_seed(seed)
        random.seed(seed)
        
        meta_train = eval(open(train_path, 'r').read())
        l_test = eval(open(test_path, 'r').read())
        meta_test = dict([(x,[]) for x in l_test]) # Need a dictionary for the above class
        
        all_train = AudioDataset(meta_train)
        test_set = AudioDataset(meta_test)
        
        # Split all_train into train + valid
        total_len = len(all_train)
        train_len = int(total_len * split_ratio)
        valid_len = total_len - train_len
        train_set, valid_set = random_split(all_train, [train_len, valid_len])
        
        self.loaderTrain = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
        self.loaderValid = DataLoader(valid_set, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
        self.loaderTest = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

In [ ]:
class CNNClassifier(nn.Module):
    def __init__(self, n_classes=N_CLASSES):
        super(CNNClassifier, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.3)
        self.fc1 = nn.Linear(32 * (N_MELS // 4) * (801 // 4), 256)
        self.fc2 = nn.Linear(256, n_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # (B, 16, mel/2, time/2)
        x = self.pool(F.relu(self.conv2(x)))  # (B, 32, mel/4, time/4)
        x = x.view(x.size(0), -1)
        x = self.dropout(F.relu(self.fc1(x)))
        return torch.sigmoid(self.fc2(x))  # multilabel → sigmoid

In [ ]:
class Pipeline():
    def __init__(self, model, learning_rate, seed = 0):
        # These two lines will (mostly) make things deterministic.
        # You're welcome to modify them to try to get a better solution.
        torch.manual_seed(seed)
        random.seed(seed)

        self.device = torch.device("cpu") # Can change this if you have a GPU, but the autograder will use CPU
        self.model = model.to(self.device) #model.cuda() # Also uncomment these lines for GPU
        self.optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
        self.criterion = nn.BCELoss()

    def evaluate(self, loader, threshold=0.5, outpath=None):
        self.model.eval()
        preds, targets, paths = [], [], []
        with torch.no_grad():
            for x, y, ps in loader:
                x = x.to(self.device) #x.cuda()
                y = y.to(self.device) #y.cuda()
                outputs = self.model(x)
                preds.append(outputs.cpu())
                targets.append(y.cpu())
                paths += list(ps)
        
        preds = torch.cat(preds)
        targets = torch.cat(targets)
        preds_bin = (preds > threshold).float()
        
        predictions = {}
        for i in range(preds_bin.shape[0]):
            predictions[paths[i]] = [TAGS[j] for j in range(len(preds_bin[i])) if preds_bin[i][j]]
        
        mAP = None
        if outpath: # Save predictions
            with open(outpath, "w") as z:
                z.write(str(predictions) + '\n')
        else: # Only compute accuracy if we're *not* saving predictions, since we can't compute test accuracy
            mAP = average_precision_score(targets, preds, average='macro')
        return predictions, mAP

    def train(self, train_loader, val_loader, num_epochs):
        for epoch in range(num_epochs):
            self.model.train()
            running_loss = 0.0
            for x, y, path in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
                x = x.to(self.device) #x.cuda()
                y = y.to(self.device) #y.cuda()
                self.optimizer.zero_grad()
                outputs = self.model(x)
                loss = self.criterion(outputs, y)
                loss.backward()
                self.optimizer.step()
                running_loss += loss.item()
            val_predictions, mAP = self.evaluate(val_loader)
            print(f"[Epoch {epoch+1}] Loss: {running_loss/len(train_loader):.4f} | Val mAP: {mAP:.4f}")

## Run everything...

In [31]:
def run1():
    model = model1()
    model.train(dataroot1 + "/train.json")
    train_preds = model.predict(dataroot1 + "/train.json")
    test_preds = model.predict(dataroot1 + "/test.json", "predictions1.json")
    
    train_labels = eval(open(dataroot1 + "/train.json").read())
    acc1 = accuracy1(train_labels, train_preds)
    print("Task 1 training accuracy = " + str(acc1))

In [114]:
def run2():
    model = model2()
    model.train(dataroot2 + "/train.json")
    train_preds = model.predict(dataroot2 + "/train.json")
    test_preds = model.predict(dataroot2 + "/test.json", "predictions2.json")
    
    train_labels = eval(open(dataroot2 + "/train.json").read())
    acc2 = accuracy2(train_labels, train_preds)
    print("Task 2 training accuracy = " + str(acc2))

In [ ]:
def run3():
    loaders = Loaders(dataroot3 + "/train.json", dataroot3 + "/test.json")
    model = CNNClassifier()
    pipeline = Pipeline(model, 1e-4)
    
    pipeline.train(loaders.loaderTrain, loaders.loaderValid, 5)
    train_preds, train_mAP = pipeline.evaluate(loaders.loaderTrain, 0.5)
    valid_preds, valid_mAP = pipeline.evaluate(loaders.loaderValid, 0.5)
    test_preds, _ = pipeline.evaluate(loaders.loaderTest, 0.5, "predictions3.json")
    
    all_train = eval(open(dataroot3 + "/train.json").read())
    for k in valid_preds:
        # We split our training set into train+valid
        # so need to remove validation instances from the training set for evaluation
        all_train.pop(k)
    acc3 = accuracy3(all_train, train_preds)
    print("Task 3 training mAP = " + str(acc3))

In [57]:
run1()

[0]	train-mlogloss:1.97826+0.00187	test-mlogloss:2.01010+0.00234


/Users/rahulbulsara/anaconda3/envs/asmt1/lib/python3.10/site-packages/xgboost/training.py:209: UserWarning: [12:49:09] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1747336884418/work/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  return getattr(self.bst, name)(*args, **kwargs)
/Users/rahulbulsara/anaconda3/envs/asmt1/lib/python3.10/site-packages/xgboost/training.py:215: UserWarning: [12:49:09] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1747336884418/work/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  self.bst.update(self.dtrain, iteration, fobj)
/Users/rahulbulsara/anaconda3/envs/asmt1/lib/python3.10/site-packages/xgboost/training.py:215: UserWarning: [12:49:10] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1747336884418/work/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  self.bst.update(self.dtrain, iteration, fobj)


[1]	train-mlogloss:1.88550+0.00224	test-mlogloss:1.94568+0.00487
[2]	train-mlogloss:1.80005+0.00279	test-mlogloss:1.88817+0.00603
[3]	train-mlogloss:1.72172+0.00261	test-mlogloss:1.83740+0.00672
[4]	train-mlogloss:1.65056+0.00314	test-mlogloss:1.79370+0.00590
[5]	train-mlogloss:1.58340+0.00353	test-mlogloss:1.74875+0.00909
[6]	train-mlogloss:1.52210+0.00327	test-mlogloss:1.70979+0.01136
[7]	train-mlogloss:1.46275+0.00332	test-mlogloss:1.67013+0.01299
[8]	train-mlogloss:1.40752+0.00289	test-mlogloss:1.63400+0.01361
[9]	train-mlogloss:1.35640+0.00338	test-mlogloss:1.60117+0.01569
[10]	train-mlogloss:1.30781+0.00446	test-mlogloss:1.56835+0.01618
[11]	train-mlogloss:1.26144+0.00355	test-mlogloss:1.53811+0.01623
[12]	train-mlogloss:1.21829+0.00370	test-mlogloss:1.50968+0.01733
[13]	train-mlogloss:1.17727+0.00345	test-mlogloss:1.48272+0.01781
[14]	train-mlogloss:1.13732+0.00337	test-mlogloss:1.45853+0.01860
[15]	train-mlogloss:1.10004+0.00342	test-mlogloss:1.43376+0.01884
[16]	train-mlogloss

/Users/rahulbulsara/anaconda3/envs/asmt1/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [12:49:25] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1747336884418/work/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Task 1 training accuracy = 0.8644628099173554


In [130]:
run2()

AttributeError: 'model2' object has no attribute 'base_xgb'

In [ ]:
run3()